# Synthesise Data with KR Model

In [1]:
import os

import chi
import chi.plots
import myokit
import numpy as np
import pandas as pd
import plotly.colors
import plotly.graph_objects as go

## Define model

In [2]:
# Define mechanistic model
directory = os.path.dirname(os.path.dirname(os.getcwd()))
mechanistic_model = chi.PharmacokineticModel(
    directory + '/models/KR_model.xml')
mechanistic_model.set_administration(compartment='central')
mechanistic_model.set_parameter_names(names={
    'myokit.bacterial_count_susceptible': 'Initial bacterial count in CFU/ml',
    'myokit.concentration_e50_adapted': 'Adapted EC 50 in ng/ml',
    'myokit.growth_rate': 'Growth rate in 1/h',
    'myokit.kappa_adapted': 'Adapted max. kill rate in 1/h',
    'myokit.kappa_susceptible': 'Wild type kill rate in ml/ng/h',
    'myokit.mutation_rate': 'Mutation rate in ml/ng/h'})
mechanistic_model.set_outputs(['myokit.total_bacterial_count'])
mechanistic_model.set_output_names({
    'myokit.total_bacterial_count': 'Bacterial count in CFU/ml'})

# Define error model
error_model = chi.LogNormalErrorModel()  # Bacterial count

# Compose model and fix not identified parameters
problem = chi.ProblemModellingController(mechanistic_model, error_model)

# Fix controlled parameters
problem.fix_parameters({
    'central.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'central.size': 1,
    'myokit.elimination_rate': 1,
    'myokit.gamma': 1})
mechanistic_model = chi.ReducedMechanisticModel(
    mechanistic_model)
mechanistic_model.fix_parameters({
    'central.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'central.size': 1,
    'myokit.elimination_rate': 1,
    'myokit.gamma': 1})

## Synthesise data

### Show example regimen

In [3]:
# Define parameters
# (parameters are motivated by MLE estimates from in vitro study)
parameters = [
    260000,  # Initial bacterial count in CFU/ml
    191,     # Adapated EC50 in ng/ml
    0.77,    # Growth rate in 1/h
    1.61,    # Adapted max. kill rate in 1/h
    0.003,   # Wild type max. kill rate in 1/h
    5E-5,    # Max. mutation rate in 1/h
    0.7      # Sigma log
]

# Define dosing regimen (dose 10 mg)
regimen = myokit.Protocol()
dose = 10E3
start_time = 4
duration = 6
period = 15
for dose_index in range(6):
    # Create dose event
    time = start_time + period * dose_index
    dose_rate = dose / duration
    dose_event = myokit.ProtocolEvent(
        level=dose_rate, start=time, duration=duration)
    regimen.add(dose_event)

# Simulate mechanistic model
times = np.linspace(0, 40, 500)
mechanistic_model.simulator.set_protocol(regimen)
result = mechanistic_model.simulate(parameters=parameters[:-1], times=times)[0]

# Sample measurements
seed = 3
measurement_times = np.linspace(0, 40, 41)
predictive_model = problem.get_predictive_model()
predictive_model._mechanistic_model.simulator.set_protocol(regimen)
measurements = predictive_model.sample(
    parameters=parameters, times=measurement_times, seed=seed)

# Create figure
fig = go.Figure()
colors = plotly.colors.qualitative.Plotly
colors = plotly.colors.qualitative.Plotly
fig.add_trace(go.Scatter(
    x=times,
    y=result,
    name='True',
    line=dict(color=colors[0])
))
fig.add_trace(go.Scatter(
    x=measurement_times,
    y=measurements['Sample'],
    name='Measured',
    mode='markers',
    marker=dict(color=colors[0], line=dict(color='black', width=1))
))
fig.update_layout(
    xaxis_title='Time in hours',
    yaxis_title='Bacterial count in CFU/ml',
    yaxis_type='log'
)
fig.show()

### Generate data for different dose levels

In [4]:
# Define measurement container
measurement_times = np.linspace(0, 40, 41)
predictive_model = problem.get_predictive_model()
measurements = pd.DataFrame(columns=
    ['ID', 'Time', 'Biomarker', 'Sample', 'Duration', 'Dose'])

# Repeat measurement process for different regimens
start_time = 4
duration = 6
period = 15
doses = [150, 300, 625, 1250, 2500, 5000, 10000]
for iteration, dose in enumerate(doses):
    # Define dosing regimen
    regimen = myokit.Protocol()
    for dose_index in range(6):
        # Create dose event
        time = start_time + period * dose_index
        dose_rate = dose / duration
        dose_event = myokit.ProtocolEvent(
            level=dose_rate, start=time, duration=duration)
        regimen.add(dose_event)

    # Sample measurements
    predictive_model._mechanistic_model.simulator.set_protocol(regimen)
    meas = predictive_model.sample(
        parameters=parameters, times=measurement_times, seed=iteration,
        include_regimen=True)
    meas['ID'] = iteration + 1
    measurements = measurements.append(meas)

# Visualise measurements
fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(measurements, meas_key='Sample')
fig.set_axis_labels(
    xlabel='Time in hours',
    ylabel='Bacterial count in CFU/ml')
fig.show()

## Export data

In [6]:
# Rename columns to standard names
temp = measurements.rename(columns={'Sample': 'Measurement'})
mask = temp['Biomarker'] == 'Bacterial count in CFU/ml'
temp.loc[mask, 'Measurement'] = temp.loc[mask, 'Measurement'].apply(int)
temp.loc[mask, 'Biomarker'] = 'Bacterial count'
temp.loc[mask, 'Biomarker unit'] = 'CFU/ml'
temp.loc[mask, 'Dose unit'] = 'ng'
temp['Time unit'] = 'h'

# Save measurements
directory = os.getcwd()
temp.to_csv(
    os.path.join(directory, 'derived_data/measurements.csv'),
    index=False)